# Scrape

### 1. Setup

The setup serves for:
- Importing libraries.
- Speficying utility functions.
- Setting up the chrome webdriver.
- Defining scraping constants.

In [3]:
# imports

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import random
import ast
import numpy as np

In [2]:
# utils

def sleep():
    time.sleep(1 + random.random())

In [3]:
# set up ChromeDriver

options = Options()
options.add_argument('--headless')  # save GUI rendering time
options.add_argument('--disable-gpu')  # prevents errors in headless mode

service = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=options)

# open base page
BASE_URL = 'https://www.migros.ch/en'
driver.get(BASE_URL)
time.sleep(5)

In [18]:
# constants

# go to subcategory
SUBCAT_XPATH = '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/mo-category-level-1-children-categories/div/ul/li[{}]/div/div/a'

# go to page
TEMPLATE_URL = '?page={}'

# hand picked from base url
CATEGORIES = [
    'fruits-vegetables',
    'bread-pastries-breakfast',
    'pasta-condiments-canned-food',
    'snacks-sweets',
    'frozen-food',
    'drinks-coffee-tea',
    'wine-beer-spirits'
]

### 2. Scrape links

Considerations and steps:
- Since all products have pages have the same template,
it is convenient separating the link retrieval and the product specifics scraping.
- In this first step we navigate to category, sub-category, and scrape all products links from the subcategories subpages.
- All links are saved a dataframe, and then permanently saved to csv. 
- The same dataframe will be used to store the product specifics in the next step.

In [9]:
# list of dictionaries 
data = []

# big giant massive loop
for cat in CATEGORIES:
    url = BASE_URL + '/category/' + cat
    driver.get(url) 

    sleep()

    # dynamically find subcategories (li[n] elements)
    subcategory_urls = []
    li_index = 1

    while True:
        sleep()
        subcat_xpath = SUBCAT_XPATH.format(li_index)
        try:
            subcategory = driver.find_element(By.XPATH, SUBCAT_XPATH)
            subcategory_url = subcategory.get_attribute('href')
            subcategory_urls.append(subcategory_url)

            print(f"Found subcategory link: {subcategory_url}")
            li_index += 1

        except NoSuchElementException:
            print("No more subcategory elements found.")
            break

    print(f"Subcategories found: {subcategory_urls}")

    for subcat_url in subcategory_urls:
        driver.get(subcat_url) 
        sleep()

        # extract products and go to the next page
        scrape_url = subcat_url + TEMPLATE_URL
        print(f"Scraping url: {scrape_url}")
        product_urls = []
        page = 1
        while True:
            sleep()
            url = scrape_url.format(page)
            driver.get(url)
            print(f"Scraping url: {url}")
            
            try:
                product_list = driver.find_elements(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/ng-component/main/div/ng-component/div/mo-items-display/div/ul/li/article/div/div[1]/a[1]')
                if not product_list:
                    print(f"No more products found on page {page}. Exiting.")
                    break
                
                for product in product_list:
                    try:
                        product_url = product.get_attribute('href')
                        product_urls.append(product_url)
                    except Exception as e:
                        print(f"Error processing product: {e}")
                        continue
                
                print(f"Finished scraping page {page}.")
                page += 1
            
            except NoSuchElementException:
                print(f"Error on page {page}, stopping.")
                break

        # store
        current_data = {
            'category': cat,
            'sub_category': subcat_url.split('/')[-1],
            'url': product_urls
        }
        data.append(current_data)

        # report 
        print(f"Finished scraping subcategory {subcat_url}")
        print(f"Data: {current_data}")

Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/fruits
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/vegetables
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/root-vegetables
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/salad
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/fresh-herbs-spices
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/ready-to-use
Found subcategory link: https://www.migros.ch/en/category/fruits-vegetables/vitamin-baskets
No more subcategory elements found.
Subcategories found: ['https://www.migros.ch/en/category/fruits-vegetables/fruits', 'https://www.migros.ch/en/category/fruits-vegetables/vegetables', 'https://www.migros.ch/en/category/fruits-vegetables/root-vegetables', 'https://www.migros.ch/en/category/fruits-vegetables/salad', 'https://www.migros.ch/en/category/fruits-vegetables/fresh-her

In [ ]:
# convert to dataframe

df = pd.DataFrame(data)
df = df.explode('url')
df.to_csv('data/product_categorization_and_urls.csv', index=False)

### 3. Scrape product information

In [4]:
df = pd.read_csv('data/products.csv')

from bs4 import BeautifulSoup

# Initialize a list to store all the product data
product_data = []

counter = 0
total = len(df)

# start from where we left off
start_index = 2337  
# big giant massive loop
for index, row in df.iloc[start_index:].iterrows():
    product_url = row['url']
    driver.get(product_url)
    sleep()  # Wait for the page to load

    try:
        # Extract the product name
        product_name = driver.find_element(By.CSS_SELECTOR, 'div.core-product-title h1').text

        # Extract the product price
        try:
            product_price = driver.find_element(By.CSS_SELECTOR, 'span.actual').text
        except Exception:
            product_price = "N/A"
            print(f"Product price not found for {product_name}")

        # Extract the product weight
        try:
            product_weight = driver.find_element(By.CSS_SELECTOR, 'span.weight-priceUnit').text
        except Exception:
            product_weight = "N/A"
            print(f"Product weight not found for {product_name}")

        # Extract the price per unit (e.g., per 100g)
        try:
            price_per_unit = driver.find_element(By.CSS_SELECTOR, 'span.listMode-priceUnit').text
        except Exception:
            price_per_unit = "N/A"
            print(f"Price per unit not found for {product_name}")

    
        ########
        # Initialize dictionary to store nutritional information
        nutrition_info = {}
        try:
            # Extract the full HTML of the table containing nutritional information
            table_html = driver.find_element(By.XPATH, '/html/body/app-root/div[1]/lsp-shop/div/div/div/div/mo-product-detail-container/main/div/mo-product-detail-page/div[1]/div[3]/mo-product-detail-information/mo-tab-layout/div/div/div[2]/mo-product-detail-nutrients-information/table').get_attribute('outerHTML')

            # Use BeautifulSoup to parse the HTML
            soup = BeautifulSoup(table_html, 'html.parser')

            # Find all rows in the table
            rows = soup.find('tbody').find_all('tr')

            # Loop through the rows and extract data
            for row in rows:
                try:
                    # Extract the nutrient name from the first column (td[1])
                    nutrient_name = row.find_all('td')[0].get_text(strip=True)

                    # Extract the nutrient value from the second column (td[2])
                    nutrient_value = row.find_all('td')[1].get_text(strip=True)

                    # Store the nutrient name and value in the dictionary
                    nutrition_info[nutrient_name] = nutrient_value

                except Exception as e:
                    print(f"Error processing row: {e}")
                    continue

        except Exception as e:
            print(f"Error extracting table.")
        ########

        # Store the scraped data in the list
        current_data = {
            'url': product_url,
            'name': product_name,
            'weight': product_weight,
            'price': product_price,
            'price_per_unit': price_per_unit,
            'nutritional_values': nutrition_info
        }
        product_data.append(current_data)

        counter += 1

        print(f"{counter}/{total}: {current_data}")

    except Exception as e:
        print(f"Error processing product at {product_url}: {str(e)}")
        continue

1/9602: {'Product URL': 'https://www.migros.ch/en/product/113382210000', 'Name': 'Schoggimoussetorte', 'Weight': '950g', 'Price': '26.–', 'Price per 100g': '2.74/100g', 'Nutritional Information': {'Energy': '1291 kJ (308 kcal)', 'Fathigh': '18 g', 'of which saturatesmedium': '3.2 g', 'Carbohydrate': '33 g', 'of which sugarshigh': '28 g', 'Protein': '3.7 g', 'Saltlow': '0.1 g'}}
2/9602: {'Product URL': 'https://www.migros.ch/en/product/113313210000', 'Name': 'Fruchttorte', 'Weight': '580g', 'Price': '14.90', 'Price per 100g': '2.57/100g', 'Nutritional Information': {'Energy': '860 kJ (205 kcal)', 'Fatmedium': '9 g', 'of which saturateslow': '0.5 g', 'Carbohydrate': '28 g', 'of which sugarsmedium': '18 g', 'Protein': '3.6 g', 'Saltlow': '0 g'}}
3/9602: {'Product URL': 'https://www.migros.ch/en/product/113317000200', 'Name': 'Himbeerrahmtorte', 'Weight': '580g', 'Price': '14.90', 'Price per 100g': '2.57/100g', 'Nutritional Information': {'Energy': '729 kJ (174 kcal)', 'Fatmedium': '8.6 g'

In [6]:
df = pd.DataFrame(product_data)
df.to_csv('data/product_specifics.csv', index=False)

df.head(10)

,Product URL,Name,Weight,Price,Price per 100g,Nutritional Information
0,https://www.migros.ch/en/product/113382210000,Schoggimoussetorte,950g,26.–,2.74/100g,"{'Energy': '1291 kJ (308 kcal)', 'Fathigh': '1..."
1,https://www.migros.ch/en/product/113313210000,Fruchttorte,580g,14.90,2.57/100g,"{'Energy': '860 kJ (205 kcal)', 'Fatmedium': '..."
2,https://www.migros.ch/en/product/113317000200,Himbeerrahmtorte,580g,14.90,2.57/100g,"{'Energy': '729 kJ (174 kcal)', 'Fatmedium': '..."
3,https://www.migros.ch/en/product/113313200200,Fruchttorte,580g,14.90,2.57/100g,"{'Energy': '842 kJ (201 kcal)', 'Fatmedium': '..."
4,https://www.migros.ch/en/product/113119805200,Muffin Schokolade,115g,1.85,1.61/100g,"{'Energy': '1831 kJ (438 kcal)', 'Fat': '23 g'..."
5,https://www.migros.ch/en/product/113052500200,Speckgugelhopf,400g,8.90,2.23/100g,"{'Energy': '1349 kJ (322 kcal)', 'Fatmedium': ..."
6,https://www.migros.ch/en/product/113656810000,Pariserring 750g,750g,22.90,3.05/100g,"{'Energy': '1327 kJ (317 kcal)', 'Fathigh': '1..."
7,https://www.migros.ch/en/product/113575700200,Erdbeerrahmtorte,950g,23.50,2.47/100g,"{'Energy': '741 kJ (177 kcal)', 'Fatmedium': '..."
8,https://www.migros.ch/en/product/113309010000,Williamstorte hell,850g,26.–,3.06/100g,"{'Energy': '867 kJ (207 kcal)', 'Fatmedium': '..."
9,https://www.migros.ch/en/product/113379410000,St. Honore Torte,750g,22.90,3.05/100g,"{'Energy': '1324 kJ (316 kcal)', 'Fathigh': '2..."


In [7]:
# close the driver
driver.quit()

### 4. Check datasets and merge

In [47]:
df1 = pd.read_csv('data/product_categorization_and_urls.csv')
df2 = pd.read_csv('data/product_specifics.csv')

# something went wrong with three links
print(df1.shape)
print(df2.shape)

(9602, 3)
(9591, 6)


In [48]:
# check for duplicates
print(df1['url'].duplicated().sum())
print(df2['url'].duplicated().sum())

# some urls are both in fruit and vegetables for example

57
57


In [49]:
# take care of nutritional_values column
df2['nutritional_values'][0]

"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g', 'of which saturates': '0 g', 'Carbohydrate': '6 g', 'of which sugars': '6 g', 'Fibre': '5 g', 'Protein': '0.6 g', 'Salt': '< 0.01 g'}"

In [50]:
# convert the strings to dictionaries
df2['nutritional_values'] = df2['nutritional_values'].apply(ast.literal_eval)

# separate columns
nutrition_df = pd.json_normalize(df2['nutritional_values'])

# merge new columns into df
df2 = df2.join(nutrition_df)

df2.head()

,url,name,weight,price,price_per_unit,nutritional_values,Energy,Fat,of which saturates,Carbohydrate,...,Selen,Caffeine,Copper,Silicon,Bicarbonate (or hydrogen carbonate),nitrate,sulfate,Chloride,Sulphate 4+,Minerals
0,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,250g,7.90,3.16/100g,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",174 kJ (42 kcal),0.6 g,0 g,6 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.migros.ch/en/product/264601302100,Dates,200g,5.95,2.98/100g,{},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.migros.ch/en/product/264500313200,Avocado,NaN,2.80,2.80/Piece,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,1.8 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,1 kg,2.95,0.30/100g,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",403 kJ (95 kcal),< 0.5 g,0.1 g,21 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,NaN,2.80,2.80/Piece,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,1.8 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# replace {} with NaN in nutritional_values
df2['nutritional_values'] = df2['nutritional_values'].apply(lambda x: np.nan if x == {} else x)
df2.head()

,url,name,weight,price,price_per_unit,nutritional_values,Energy,Fat,of which saturates,Carbohydrate,...,Selen,Caffeine,Copper,Silicon,Bicarbonate (or hydrogen carbonate),nitrate,sulfate,Chloride,Sulphate 4+,Minerals
0,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,250g,7.90,3.16/100g,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",174 kJ (42 kcal),0.6 g,0 g,6 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.migros.ch/en/product/264601302100,Dates,200g,5.95,2.98/100g,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.migros.ch/en/product/264500313200,Avocado,NaN,2.80,2.80/Piece,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,1.8 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,1 kg,2.95,0.30/100g,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",403 kJ (95 kcal),< 0.5 g,0.1 g,21 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,NaN,2.80,2.80/Piece,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,1.8 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# sanity check
print(len(df2.columns))
print(len(df2.columns.unique()))
print(len(df2))

70
70
9591


In [53]:
# merge the two dataframes
df = pd.merge(df1, df2, on='url', how='left')
len(df)

9716

In [54]:
# drop duplicates
df = df.drop_duplicates(subset='url')
len(df)

9545

In [55]:
# check df
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9545 entries, 0 to 9715
Data columns (total 72 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   category                                    9545 non-null   object
 1   sub_category                                9545 non-null   object
 2   url                                         9545 non-null   object
 3   name                                        9534 non-null   object
 4   weight                                      9036 non-null   object
 5   price                                       9501 non-null   object
 6   price_per_unit                              9440 non-null   object
 7   nutritional_values                          7770 non-null   object
 8   Energy                                      7681 non-null   object
 9   Fat                                         1976 non-null   object
 10  of which saturates           

In [56]:
df['weight'].unique()

array(['250g', '200g', nan, '1 kg', '500g', '3 Stück', '1kg', '1 Stück',
       '750g', '300g', '50g', '3kg', '100g', '2kg', '240g', '400g',
       '100 g', '1 Kilo', '1.5kg', '700g', '2000g', '1500g', '1 Stk.',
       '850g', '600g', '2.5kg', '350g', '150g', '440g', '330g', '160g',
       '1000g', '40g', '2 Stück', '475g', '75g', '800g', '125g', '480g',
       '5kg', '500ml', '210g', '130g', '1000ml', '2 x 500ml', '60ml',
       '120g', '80g', '270ml', '180g', '230g', '1l', '220g', '320g',
       '140g', '2 x 130g', '260g', '375g', '30g', '10g', '35g', '20g',
       '25g', '380g', '460g', '370g', '540g', '4dl', '470g', '410g',
       '2.5dl', '3200g', '3100g', '360g', '650g', '45g', '105g', '135g',
       '270g', '60g', '90g', '110g', '70g', '85g', '65g', '93g', '114g',
       '215g', '68g', '2 x 146g', '95g', '3 x 80g', '82g', '2x100g',
       '46g', '285g', '138g', '58g', '53g', '115g', '280g', '170g', '71g',
       '2 x 100g', '4 x 80g', '420g', '450g', '55g', '275g', '83g',
      

In [57]:
import pandas as pd
import numpy as np
import re

# Define unit mapping to standardize units
unit_mapping = {
    'kilo': 'kg',
    'kilogram': 'kg',
    'kilograms': 'kg',
    'kgs': 'kg',
    'kg': 'kg',
    'g': 'g',
    'gram': 'g',
    'grams': 'g',
    'mg': 'mg',
    'µg': 'µg',
    'l': 'l',
    'litre': 'l',
    'litres': 'l',
    'dl': 'dl',
    'cl': 'cl',
    'ml': 'ml',
    'piece': 'piece',
    'pieces': 'piece',
    'stück': 'piece',
    'stk': 'piece',
    'stk.': 'piece',
    'tabl': 'tablet',
    'tabl.': 'tablet',
    'tab': 'tablet',
    'tabs': 'tablet',
    'balls': 'piece',
    'g.': 'g',
    'kg.': 'kg',
    'ml.': 'ml',
    'l.': 'l',
}

# Define the parsing function
def parse_weight_entry(entry):
    # Initialize default values
    quantity = 1
    weight_per_unit = np.nan
    weight_unit = np.nan

    if pd.isnull(entry):
        return pd.Series([quantity, weight_per_unit, weight_unit])

    entry = str(entry).strip()

    # Handle entries with 'Stück', 'Stk.', 'piece', 'Tabl.', 'Balls', 'Stk'
    piece_pattern = r'(?i)^(\d+)\s*(Stück|Stk\.?|Stk|Pieces?|Piece|Tabl\.?|Balls?)$'
    piece_match = re.match(piece_pattern, entry)
    if piece_match:
        quantity = int(piece_match.group(1))
        weight_unit = 'piece'
        return pd.Series([quantity, weight_per_unit, weight_unit])

    # Handle entries with 'x' indicating multiple units, e.g., '2 x 500ml', '3x90g', '2x100g'
    x_pattern = r'^(\d+)\s*[xX]\s*([\d\.,]+)\s*([a-zA-Zµ\.]+)'
    x_match = re.match(x_pattern, entry)
    if x_match:
        quantity = int(x_match.group(1))
        weight_per_unit = float(x_match.group(2).replace(',', '.'))
        weight_unit = x_match.group(3).lower().replace('.', '')
        if weight_unit in unit_mapping:
            weight_unit = unit_mapping[weight_unit]
        return pd.Series([quantity, weight_per_unit, weight_unit])

    # Handle entries like '1 kg', '500g', '750ml', including '1kg', '100 g', '500 g'
    weight_pattern = r'^([\d\.,]+)\s*([a-zA-Zµ\.]+)'
    weight_match = re.match(weight_pattern, entry)
    if weight_match:
        weight_per_unit = float(weight_match.group(1).replace(',', '.'))
        weight_unit = weight_match.group(2).lower().replace('.', '')
        if weight_unit in unit_mapping:
            weight_unit = unit_mapping[weight_unit]
        return pd.Series([quantity, weight_per_unit, weight_unit])

    # Handle special cases like '1 Kilo', '1.5kg'
    special_pattern = r'^([\d\.,]+)\s*(kg|kilo|g|mg|µg|ml|l|dl|cl)'
    special_match = re.match(special_pattern, entry, re.IGNORECASE)
    if special_match:
        weight_per_unit = float(special_match.group(1).replace(',', '.'))
        weight_unit = special_match.group(2).lower()
        if weight_unit in unit_mapping:
            weight_unit = unit_mapping[weight_unit]
        return pd.Series([quantity, weight_per_unit, weight_unit])

    # Handle entries like '6 x 1,5l' (comma as decimal separator)
    comma_x_pattern = r'^(\d+)\s*[xX]\s*([\d\.,]+)\s*([a-zA-Zµ\.]+)'
    comma_x_match = re.match(comma_x_pattern, entry)
    if comma_x_match:
        quantity = int(comma_x_match.group(1))
        weight_per_unit = float(comma_x_match.group(2).replace(',', '.'))
        weight_unit = comma_x_match.group(3).lower().replace('.', '')
        if weight_unit in unit_mapping:
            weight_unit = unit_mapping[weight_unit]
        return pd.Series([quantity, weight_per_unit, weight_unit])

    # If none of the patterns match, return defaults
    return pd.Series([quantity, weight_per_unit, weight_unit])

# Apply the parsing function to df["weight"]
df[['quantity', 'weight_per_unit', 'weight_unit']] = df['weight'].apply(parse_weight_entry)

# Function to convert weight to base unit and record the base unit
def convert_to_base_unit(row):
    weight = row['weight_per_unit']
    unit = row['weight_unit']
    if pd.isnull(weight) or pd.isnull(unit):
        return pd.Series([np.nan, np.nan])
    if unit == 'kg':
        base_weight = weight * 1000        # Convert kg to g
        base_unit = 'g'
    elif unit == 'g':
        base_weight = weight               # Already in grams
        base_unit = 'g'
    elif unit == 'mg':
        base_weight = weight / 1000        # Convert mg to g
        base_unit = 'g'
    elif unit == 'µg':
        base_weight = weight / 1e6         # Convert µg to g
        base_unit = 'g'
    elif unit == 'l':
        base_weight = weight * 1000        # Convert liters to ml
        base_unit = 'ml'
    elif unit == 'dl':
        base_weight = weight * 100         # Convert deciliters to ml
        base_unit = 'ml'
    elif unit == 'cl':
        base_weight = weight * 10          # Convert centiliters to ml
        base_unit = 'ml'
    elif unit == 'ml':
        base_weight = weight               # Already in milliliters
        base_unit = 'ml'
    else:
        base_weight = weight               # For units like 'piece' or 'tablet'
        base_unit = unit                   # Keep the original unit
    return pd.Series([base_weight, base_unit])

# Apply the conversion function to create 'weight_in_base_unit' and 'base_unit' columns
df[['weight_in_base_unit', 'base_unit']] = df.apply(convert_to_base_unit, axis=1)

# Drop the original 'weight' column
df = df.drop(columns=['weight'])

# Display the first few rows of the DataFrame
df.head()


,category,sub_category,url,name,price,price_per_unit,nutritional_values,Energy,Fat,of which saturates,...,nitrate,sulfate,Chloride,Sulphate 4+,Minerals,quantity,weight_per_unit,weight_unit,weight_in_base_unit,base_unit
0,fruits-vegetables,fruits,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,7.90,3.16/100g,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",174 kJ (42 kcal),0.6 g,0 g,...,NaN,NaN,NaN,NaN,NaN,1.0,250.0,g,250.0,g
1,fruits-vegetables,fruits,https://www.migros.ch/en/product/264601302100,Dates,5.95,2.98/100g,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,200.0,g,200.0,g
2,fruits-vegetables,fruits,https://www.migros.ch/en/product/264500313200,Avocado,2.80,2.80/Piece,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,fruits-vegetables,fruits,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,2.95,0.30/100g,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",403 kJ (95 kcal),< 0.5 g,0.1 g,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,kg,1000.0,g
5,fruits-vegetables,fruits,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,2.80,2.80/Piece,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [58]:
df["weight_unit"].unique()

array(['g', nan, 'kg', 'piece', 'ml', 'l', 'dl', 'cl'], dtype=object)

In [59]:
df["weight_per_unit"].unique()

array([2.500e+02, 2.000e+02,       nan, 1.000e+00, 5.000e+02, 7.500e+02,
       3.000e+02, 5.000e+01, 3.000e+00, 1.000e+02, 2.000e+00, 2.400e+02,
       4.000e+02, 1.500e+00, 7.000e+02, 2.000e+03, 1.500e+03, 8.500e+02,
       6.000e+02, 2.500e+00, 3.500e+02, 1.500e+02, 4.400e+02, 3.300e+02,
       1.600e+02, 1.000e+03, 4.000e+01, 4.750e+02, 7.500e+01, 8.000e+02,
       1.250e+02, 4.800e+02, 5.000e+00, 2.100e+02, 1.300e+02, 6.000e+01,
       1.200e+02, 8.000e+01, 2.700e+02, 1.800e+02, 2.300e+02, 2.200e+02,
       3.200e+02, 1.400e+02, 2.600e+02, 3.750e+02, 3.000e+01, 1.000e+01,
       3.500e+01, 2.000e+01, 2.500e+01, 3.800e+02, 4.600e+02, 3.700e+02,
       5.400e+02, 4.000e+00, 4.700e+02, 4.100e+02, 3.200e+03, 3.100e+03,
       3.600e+02, 6.500e+02, 4.500e+01, 1.050e+02, 1.350e+02, 9.000e+01,
       1.100e+02, 7.000e+01, 8.500e+01, 6.500e+01, 9.300e+01, 1.140e+02,
       2.150e+02, 6.800e+01, 1.460e+02, 9.500e+01, 8.200e+01, 4.600e+01,
       2.850e+02, 1.380e+02, 5.800e+01, 5.300e+01, 

In [60]:
df["quantity"].unique()

array([1.0e+00, 3.0e+00, 2.0e+00, 4.0e+00, 6.0e+00, 5.0e+00, 5.0e+02,
       2.2e+02, 1.3e+03, 1.2e+01, 1.6e+01, 8.0e+00, 1.0e+01, 5.0e+01,
       2.0e+01, 7.0e+00, 9.0e+00, 2.4e+01, 1.8e+01])

In [61]:
len(df)

9545

In [62]:
# parse price column to numeric
print(df['price'].unique())

# # remove dash '-' at end of string if present
df['price'] = df['price'].str.replace('–', '')
print(df['price'].unique())

# parse price column to numeric now that all values are numeric
df['price'] = df['price'].astype(float)
df['price'].unique()

['7.90' '5.95' '2.80' '2.95' '6.30' '2.50' '3.95' '8.90' '3.70' '3.80'
 '4.50' '3.30' '3.50' '0.95' '5.50' '4.80' '4.95' '3.75' '6.95' '2.20'
 '3.20' '4.70' '2.70' '3.10' '7.30' '5.70' '4.30' '2.60' '5.60' '5.40'
 '2.90' '4.20' '1.40' '9.50' '7.50' '6.50' '5.80' '6.70' '6.90' '5.30'
 '1.85' '5.90' '5.85' '1.–' '1.50' '0.60' '5.–' '1.90' '3.–' '1.80' '9.60'
 '0.90' '2.30' '3.60' '0.50' '4.40' '6.40' '3.90' '1.20' '4.90' '4.65'
 '2.75' '1.30' '0.55' '9.90' '6.25' '11.30' '2.55' '9.95' '2.–' '3.40'
 '8.95' '2.40' '2.85' '4.55' '1.65' '1.10' '5.20' '8.50' '8.80' '7.80'
 '9.40' '4.60' '5.65' '7.95' '2.25' '2.15' '11.95' '1.60' '4.10' '4.15'
 '1.95' '3.65' '2.65' '6.60' '2.10' '7.40' '0.70' '3.85' '6.20' '5.75'
 '3.55' '3.05' '3.35' '4.45' '1.70' '11.40' nan '12.95' '11.50' '14.30'
 '11.20' '0.75' '5.10' '4.–' '1.25' '0.85' '6.–' '0.80' '15.–' '3.25'
 '4.05' '6.10' '4.25' '3.15' '1.15' '15.90' '3.45' '1.75' '1.35' '6.80'
 '14.90' '18.60' '9.20' '8.20' '12.90' '32.–' '13.50' '30.–' '8.40'
 '1

array([  7.9 ,   5.95,   2.8 ,   2.95,   6.3 ,   2.5 ,   3.95,   8.9 ,
         3.7 ,   3.8 ,   4.5 ,   3.3 ,   3.5 ,   0.95,   5.5 ,   4.8 ,
         4.95,   3.75,   6.95,   2.2 ,   3.2 ,   4.7 ,   2.7 ,   3.1 ,
         7.3 ,   5.7 ,   4.3 ,   2.6 ,   5.6 ,   5.4 ,   2.9 ,   4.2 ,
         1.4 ,   9.5 ,   7.5 ,   6.5 ,   5.8 ,   6.7 ,   6.9 ,   5.3 ,
         1.85,   5.9 ,   5.85,   1.  ,   1.5 ,   0.6 ,   5.  ,   1.9 ,
         3.  ,   1.8 ,   9.6 ,   0.9 ,   2.3 ,   3.6 ,   0.5 ,   4.4 ,
         6.4 ,   3.9 ,   1.2 ,   4.9 ,   4.65,   2.75,   1.3 ,   0.55,
         9.9 ,   6.25,  11.3 ,   2.55,   9.95,   2.  ,   3.4 ,   8.95,
         2.4 ,   2.85,   4.55,   1.65,   1.1 ,   5.2 ,   8.5 ,   8.8 ,
         7.8 ,   9.4 ,   4.6 ,   5.65,   7.95,   2.25,   2.15,  11.95,
         1.6 ,   4.1 ,   4.15,   1.95,   3.65,   2.65,   6.6 ,   2.1 ,
         7.4 ,   0.7 ,   3.85,   6.2 ,   5.75,   3.55,   3.05,   3.35,
         4.45,   1.7 ,  11.4 ,    nan,  12.95,  11.5 ,  14.3 ,  11.2 ,
      

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9545 entries, 0 to 9715
Data columns (total 76 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   category                                    9545 non-null   object 
 1   sub_category                                9545 non-null   object 
 2   url                                         9545 non-null   object 
 3   name                                        9534 non-null   object 
 4   price                                       9501 non-null   float64
 5   price_per_unit                              9440 non-null   object 
 6   nutritional_values                          7770 non-null   object 
 7   Energy                                      7681 non-null   object 
 8   Fat                                         1976 non-null   object 
 9   of which saturates                          1815 non-null   object 
 10  Carbohydrate     

In [64]:
# price per 100g col -> create price_unit and price_per_unit columns
# split on "/"

# print all of them to see what we are working with
for up in df['price_per_unit'].unique():
    print(up)

# split on "/"
df[['price_per_unit', 'price_unit']] = df['price_per_unit'].str.split('/', expand=True)
# for up in df['price_unit'].unique():
#     print(up)

# remove dash '-' at end of string if present
df['price_per_unit'] = df['price_per_unit'].str.replace('–', '')
print(df['price_per_unit'].unique())

# parse to float
df['price_per_unit'] = df['price_per_unit'].astype(float)

print(df['price_unit'].unique())

# Change cols with "piece" to "Piece"
df['price_unit'] = df['price_unit'].str.replace('piece', 'Piece')
print(df['price_unit'].unique())

3.16/100g
2.98/100g
2.80/Piece
0.30/100g
0.63/100g
0.50/100g
1.58/100g
0.83/piece
1.78/100g
0.25/100g
5.95/piece
0.74/100g
3.95/piece
0.76/100g
0.89/100g
0.90/100g
0.22/100g
1.–/100g
5.95/Piece
0.70/100g
0.95/piece
0.55/100g
2.80/piece
0.37/100g
0.96/100g
1.65/100g
0.75/100g
1.48/100g
1.80/100g
0.35/100g
4.40/100g
0.66/100g
2.78/100g
0.15/100g
3.20/piece
0.94/100g
0.54/100g
0.31/100g
7.30/100g
2.20/100g
0.29/100g
4.30/piece
2.60/piece
2.24/100g
0.36/100g
1.21/100g
0.84/100g
1.40/100g
0.95/100g
3.–/100g
2.60/100g
1.16/100g
2.38/100g
1.68/100g
2.30/100g
2.12/100g
1.92/100g
1.85/100g
0.99/100g
0.59/100g
1.–/Piece
1.50/100g
2.80/100g
0.60/piece
3.20/Piece
0.38/100g
0.60/100g
1.83/100g
3.60/100g
1.–/piece
2.60/Piece
0.60/Piece
3.80/piece
1.18/100g
0.90/piece
0.87/piece
1.50/Piece
1.50/piece
0.46/100g
3.60/piece
0.42/100g
4.50/Piece
0.40/100g
0.50/Piece
3.95/Piece
5.60/Piece
2.20/piece
3.10/piece
0.33/100g
4.40/Piece
5.80/100g
0.69/100g
0.64/100g
0.39/100g
1.20/piece
1.20/Piece
4.40/piece
0.

In [65]:
df.head()

,category,sub_category,url,name,price,price_per_unit,nutritional_values,Energy,Fat,of which saturates,...,sulfate,Chloride,Sulphate 4+,Minerals,quantity,weight_per_unit,weight_unit,weight_in_base_unit,base_unit,price_unit
0,fruits-vegetables,fruits,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,7.90,3.16,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",174 kJ (42 kcal),0.6 g,0 g,...,NaN,NaN,NaN,NaN,1.0,250.0,g,250.0,g,100g
1,fruits-vegetables,fruits,https://www.migros.ch/en/product/264601302100,Dates,5.95,2.98,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,200.0,g,200.0,g,100g
2,fruits-vegetables,fruits,https://www.migros.ch/en/product/264500313200,Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Piece
4,fruits-vegetables,fruits,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,2.95,0.30,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",403 kJ (95 kcal),< 0.5 g,0.1 g,...,NaN,NaN,NaN,NaN,1.0,1.0,kg,1000.0,g,100g
5,fruits-vegetables,fruits,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",643 kJ (156 kcal),14 g,2 g,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Piece


In [66]:
# Energy column

# Function to split the Energy column into energy_kJ and energy_kcal
def split_energy(energy):
    if pd.isna(energy) or energy.strip() == '':
        return np.nan, np.nan
    try:
        # Extract the kJ value (before the first space)
        energy_kJ = energy.split(' ')[0]
        # Extract the kcal value (inside the parentheses)
        energy_kcal = energy.split('(')[1].replace('kcal)', '').strip()
        return float(energy_kJ), float(energy_kcal)
    except (IndexError, ValueError):
        return np.nan, np.nan

# Apply the function to split the Energy column
df[['energy_kJ', 'energy_kcal']] = df['Energy'].apply(lambda x: pd.Series(split_energy(x)))

# Drop the original Energy column
df.drop(columns=['Energy'], inplace=True)

# Display the result
df.head()


,category,sub_category,url,name,price,price_per_unit,nutritional_values,Fat,of which saturates,Carbohydrate,...,Sulphate 4+,Minerals,quantity,weight_per_unit,weight_unit,weight_in_base_unit,base_unit,price_unit,energy_kJ,energy_kcal
0,fruits-vegetables,fruits,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,7.90,3.16,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",0.6 g,0 g,6 g,...,NaN,NaN,1.0,250.0,g,250.0,g,100g,174.0,42.0
1,fruits-vegetables,fruits,https://www.migros.ch/en/product/264601302100,Dates,5.95,2.98,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,200.0,g,200.0,g,100g,NaN,NaN
2,fruits-vegetables,fruits,https://www.migros.ch/en/product/264500313200,Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",14 g,2 g,1.8 g,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Piece,643.0,156.0
4,fruits-vegetables,fruits,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,2.95,0.30,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",< 0.5 g,0.1 g,21 g,...,NaN,NaN,1.0,1.0,kg,1000.0,g,100g,403.0,95.0
5,fruits-vegetables,fruits,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",14 g,2 g,1.8 g,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Piece,643.0,156.0


In [67]:
cols_to_parse = ['Fat', 'of which saturates',
       'Carbohydrate', 'of which sugars', 'Fibre', 'Protein', 'Salt', 'Fatlow',
       'of which saturateslow', 'of which sugarsmedium', 'Saltlow',
       'of which sugarshigh', 'Saltmedium', 'of which sugarslow', 'Salthigh',
       'of which mono-unsaturates', 'Fathigh', 'of which saturatesmedium',
       'Fatmedium', 'of which saturateshigh', 'Sodium', 'Energy value in kJ',
       'Energy value in kcal', 'Vitamin B1 (thiamin)',
       'Vitamin B2 (riboflavin)', 'Niacin', 'Vitamin B6',
       'folic acid / folate', 'Iron', 'Magnesium',
       'of which polyvalents alcohols', 'Phosphorus', 'Mangan', 'Vitamin B12',
       'Vitamin D', 'Pantothenic acid', 'Calcium', 'folic acid', 'Vitamin E',
       'Vitamin K', 'Vitamin C', 'Biotin', 'Omega-3 fatty acids', 'Potassium',
       'Zinc', 'Alpha linolenic acid (omega-3 fatty acids)', 'Water',
       'Omega-6 fatty acids', 'Iodine', 'Fluoride', 'of which starch',
       'Vitamin A', 'Cholesterin', 'Selen', 'Caffeine', 'Copper', 'Silicon',
       'Bicarbonate (or hydrogen carbonate)', 'nitrate', 'sulfate', 'Chloride',
       'Sulphate 4+', 'Minerals']

In [68]:
# Function to parse the weight values
def parse_value(value):
    if pd.isna(value) or value.strip() == '':
        return np.nan
    try:
        # Remove leading '~', '<' and other characters that aren't part of the numeric value
        value = value.replace('~', '').replace('<', '').strip()
        # Extract the numeric part
        numeric_value = ''.join([char for char in value if char.isdigit() or char == '.' or char == ','])
        # Convert to float (handle European comma notation)
        return float(numeric_value.replace(',', '.'))
    except (ValueError, TypeError):
        return np.nan

# Apply the function to parse the numeric part of the values
for col in cols_to_parse:
    df[col] = df[col].apply(parse_value)
    df.rename(columns={col: col + ' (g)'}, inplace=True)

# Display the result
df.head()

,category,sub_category,url,name,price,price_per_unit,nutritional_values,Fat (g),of which saturates (g),Carbohydrate (g),...,Sulphate 4+ (g),Minerals (g),quantity,weight_per_unit,weight_unit,weight_in_base_unit,base_unit,price_unit,energy_kJ,energy_kcal
0,fruits-vegetables,fruits,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,7.90,3.16,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",0.6,0.0,6.0,...,NaN,NaN,1.0,250.0,g,250.0,g,100g,174.0,42.0
1,fruits-vegetables,fruits,https://www.migros.ch/en/product/264601302100,Dates,5.95,2.98,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,200.0,g,200.0,g,100g,NaN,NaN
2,fruits-vegetables,fruits,https://www.migros.ch/en/product/264500313200,Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",14.0,2.0,1.8,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Piece,643.0,156.0
4,fruits-vegetables,fruits,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,2.95,0.30,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",0.5,0.1,21.0,...,NaN,NaN,1.0,1.0,kg,1000.0,g,100g,403.0,95.0
5,fruits-vegetables,fruits,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",14.0,2.0,1.8,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,Piece,643.0,156.0


In [69]:
# convert cols to strings
df['category'] = df['category'].astype(str)
df['sub_category'] = df['sub_category'].astype(str)
df['url'] = df['url'].astype(str)
df['name'] = df['name'].astype(str)
df['price_unit'] = df['price_unit'].astype(str)
df['weight_unit'] = df['weight_unit'].astype(str)

In [73]:

# Display the result
df.head()

,category,sub_category,url,name,total_price,price_per_unit,nutritional_values,Fat (g),of which saturates (g),Carbohydrate (g),...,Sulphate 4+ (g),Minerals (g),quantity,weight_per_unit,weight_unit,weight_in_base_unit,base_unit,price_unit,energy_kJ,energy_kcal
0,fruits-vegetables,fruits,https://www.migros.ch/en/product/265680802400,Migros Bio · blueberries,7.90,3.16,"{'Energy': '174 kJ (42 kcal)', 'Fat': '0.6 g',...",0.6,0.0,6.0,...,NaN,NaN,1.0,250.0,g,250.0,g,100g,174.0,42.0
1,fruits-vegetables,fruits,https://www.migros.ch/en/product/264601302100,Dates,5.95,2.98,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,200.0,g,200.0,g,100g,NaN,NaN
2,fruits-vegetables,fruits,https://www.migros.ch/en/product/264500313200,Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",14.0,2.0,1.8,...,NaN,NaN,1.0,NaN,nan,NaN,NaN,Piece,643.0,156.0
4,fruits-vegetables,fruits,https://www.migros.ch/en/product/264280101000,Bio Fairtrade · Bananas,2.95,0.30,"{'Energy': '403 kJ (95 kcal)', 'Fat': '< 0.5 g...",0.5,0.1,21.0,...,NaN,NaN,1.0,1.0,kg,1000.0,g,100g,403.0,95.0
5,fruits-vegetables,fruits,https://www.migros.ch/en/product/264580113200,Migros Bio · Avocado,2.80,2.80,"{'Energy': '643 kJ (156 kcal)', 'Fat': '14 g',...",14.0,2.0,1.8,...,NaN,NaN,1.0,NaN,nan,NaN,NaN,Piece,643.0,156.0


In [72]:
# rename price to total_price
df.rename(columns={'price': 'total_price'}, inplace=True)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9545 entries, 0 to 9715
Data columns (total 78 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   category                                        9545 non-null   object 
 1   sub_category                                    9545 non-null   object 
 2   url                                             9545 non-null   object 
 3   name                                            9545 non-null   object 
 4   total_price                                     9501 non-null   float64
 5   price_per_unit                                  9440 non-null   float64
 6   nutritional_values                              7770 non-null   object 
 7   Fat (g)                                         1965 non-null   float64
 8   of which saturates (g)                          1804 non-null   float64
 9   Carbohydrate (g)                              

In [76]:
# reorder columns: name, url, category, sub_category, total_price, price_per_unit, price_unit, quantity, weight_per_unit, weight_unit, weight_in_base_unit, base_unit, energy_kJ, energy_kcal, Fat (g), of which saturates (g), Carbohydrate (g), of which sugars (g), Fibre (g), Protein (g), Salt (g), Fatlow (g), of which saturateslow (g), of which sugarsmedium (g), Saltlow (g), of which sugarshigh (g), Saltmedium (g), of which sugarslow (g), Salthigh (g), of which mono-unsaturates (g), Fathigh (g), of which saturatesmedium (g), Fatmedium (g), of which saturateshigh (g), Sodium (g), Energy value in kJ (g), Energy value in kcal (g), Vitamin B1 (thiamin) (g), Vitamin B2 (riboflavin) (g), Niacin (g), Vitamin B6 (g), folic acid / folate (g), Iron (g), Magnesium (g), of which polyvalents alcohols (g), Phosphorus (g), Mangan (g), Vitamin B12 (g), Vitamin D (g), Pantothenic acid (g), Calcium (g), folic acid (g), Vitamin E (g), Vitamin K (g), Vitamin C (g), Biotin (g), Omega-3 fatty acids (g), Potassium (g), Zinc (g), Alpha linolenic acid (omega-3 fatty acids) (g), Water (g), Omega-6 fatty acids (g), Iodine (g), Fluoride (g), of which starch (g), Vitamin A (g), Cholesterin (g), Selen (g), Caffeine (g), Copper (g), Silicon (g), Bicarbonate (or hydrogen carbonate) (g), nitrate (g), sulfate (g), Chloride (g), Sulphate 4+ (g), Minerals (g)

df = df[['name', 'url', 'category', 'sub_category', 'total_price', 'price_per_unit', 'price_unit', 'quantity', 'weight_per_unit', 'weight_unit', 'weight_in_base_unit', 'base_unit', 'energy_kJ', 'energy_kcal', 'Fat (g)', 'of which saturates (g)', 'Carbohydrate (g)', 'of which sugars (g)', 'Fibre (g)', 'Protein (g)', 'Salt (g)', 'Fatlow (g)', 'of which saturateslow (g)', 'of which sugarsmedium (g)', 'Saltlow (g)', 'of which sugarshigh (g)', 'Saltmedium (g)', 'of which sugarslow (g)', 'Salthigh (g)', 'of which mono-unsaturates (g)', 'Fathigh (g)', 'of which saturatesmedium (g)', 'Fatmedium (g)', 'of which saturateshigh (g)', 'Sodium (g)', 'Energy value in kJ (g)', 'Energy value in kcal (g)', 'Vitamin B1 (thiamin) (g)', 'Vitamin B2 (riboflavin) (g)', 'Niacin (g)', 'Vitamin B6 (g)', 'folic acid / folate (g)', 'Iron (g)', 'Magnesium (g)', 'of which polyvalents alcohols (g)', 'Phosphorus (g)', 'Mangan (g)', 'Vitamin B12 (g)', 'Vitamin D (g)', 'Pantothenic acid (g)', 'Calcium (g)', 'folic acid (g)', 'Vitamin E (g)', 'Vitamin K (g)', 'Vitamin C (g)', 'Biotin (g)', 'Omega-3 fatty acids (g)', 'Potassium (g)', 'Zinc (g)', 'Alpha linolenic acid (omega-3 fatty acids) (g)', 'Water (g)', 'Omega-6 fatty acids (g)', 'Iodine (g)', 'Fluoride (g)', 'of which starch (g)', 'Vitamin A (g)', 'Cholesterin (g)', 'Selen (g)', 'Caffeine (g)', 'Copper (g)', 'Silicon (g)', 'Bicarbonate (or hydrogen carbonate) (g)', 'nitrate (g)', 'sulfate (g)', 'Chloride (g)', 'Sulphate 4+ (g)', 'Minerals (g)']] 

In [77]:
# save final dataframe
df.to_csv('data/products_complete_info.csv', index=False)